In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%matplotlib inline
import sys
sys.path.append('../../')

import tensorflow as tf
# tf.enable_eager_execution()
from tensorflow import gfile
import numpy as np
from tgs import data
import time
from matplotlib import pyplot as plt

np.set_printoptions(suppress=True,linewidth=np.nan,threshold=np.nan)

In [ ]:
config_dict = {
    'train_file_pattern': '/home/jfaath/projects/tgs/data/train/tfrecord/tgs*',
    'valid_file_pattern': '/home/jfaath/projects/tgs/data/valid/tfrecord/tgs*',
    'test_file_pattern': '/home/jfaath/projects/tgs/data/test/tfrecord/tgs*',
    'shuf_buf': 50000,
    'parallel_calls': 8,
    'ext': {
        'resize_dim': 128,
        'resize_method': 'resize'
    }
}

batch_size = 20
num_epochs = 1

def run_data(idi, mode=tf.estimator.ModeKeys.TRAIN, iters=None, resize_mth='pad', augment=None):
    if iters is None:
        iters = 999999
    
    sess = tf.InteractiveSession()
    d = idi.input_fn(mode, augment)
    cnt = 0
    ids = []
    imgs = []
    masks = []
    resizes = []
    while True:
        try:
            img_dict, mask = sess.run(d)
            ids.extend(img_dict['id'])
            imgs.extend(img_dict['img'])
            masks.extend(mask)
            resizes.extend(img_dict[resize_mth])
            cnt += 1
            if cnt > iters:
                break
        except tf.errors.OutOfRangeError:
            print(f"Epoch finished on iteration {cnt}")
            break

    sess.close()
    tf.reset_default_graph()
    
    return ids, np.asarray(imgs), np.asarray(masks), np.asarray(resizes)


def plot_images_and_masks(ids, imgs, masks, count=4):
    fig, axes = plt.subplots(nrows=count, ncols=2, figsize=(10, 10 * (count // 2)), sharex=True, sharey=True)
    ax = axes.ravel()

    for i in range(count):
        ax[i * 2].imshow(imgs[i], cmap=plt.cm.gray, interpolation='none', aspect='auto')
        ax[i * 2].set_title(ids[i].decode())
        ax[i * 2 + 1].imshow(masks[i], cmap=plt.cm.gray, interpolation='none', aspect='auto')


In [ ]:
%%time
augment = {'flip': None, 'rotation': 10}
# augment = None
idi = data.ImageDataInput(config_dict,
                          batch_size=batch_size,
                          num_epochs=num_epochs,
                          preprocess=False)
resize_mth = config_dict['ext']['resize_method']
ids, imgs, masks, resizes = run_data(idi, iters=None, 
                                     mode=tf.estimator.ModeKeys.PREDICT, resize_mth=resize_mth, augment=augment)

In [ ]:
plot_images_and_masks(ids, imgs, masks, count=10)

In [ ]:
idx = 0

print('IMAGES')
print(f'shape: {imgs[idx].shape}, dtype: {imgs[idx].dtype}')
print(f'min: {np.min(imgs[idx])}, max: {np.max(imgs[idx])}')
print('MASKS')
print(f'shape: {masks[idx].shape}, dtype: {masks[idx].dtype}')
print(f'min: {np.min(masks[1])}, max: {np.max(masks[1])}')

In [ ]:
cnt = 2
images_pre = imgs[0:cnt]
masks_pre = masks[0:cnt]

transforms = [[1., 0., 0., 0., 1., 0., 0.001, 0.001]]

imgt = tf.contrib.image.transform(images_pre, transforms, interpolation='NEAREST')
maskt = tf.contrib.image.transform(tf.expand_dims(masks_pre, axis=-1), transforms, interpolation='NEAREST')
maskt = tf.squeeze(maskt, axis=-1)

with tf.Session() as s:
    imgtt, masktt = s.run([imgt, maskt])
    
tf.reset_default_graph()

In [ ]:
plot_images_and_masks(ids, imgtt, masktt, count=cnt)